In [1]:
cd C:\MIT\Lab\LinsenChong\SANDAG

C:\MIT\Lab\LinsenChong\SANDAG


In [2]:
import pandas as pd
import numpy as np
import sqlite3, os

In [3]:
sqlite_file='C:\MIT\Lab\LinsenChong\SANDAG\Database\TSS_SANDAG_NETWORK_V8_typicalDayLastestNetwork_8-10AM_1.sqlite'

In [7]:
db=sqlite3.connect(sqlite_file)

In [29]:
tableListQuery="SELECT name FROM sqlite_master WHERE type='table' ORDER BY Name"
cursor = db.cursor() 
cursor.execute(tableListQuery)
tables = map(lambda t: t[0], cursor.fetchall())
print tables #get tables

[u'DETEQUIPVEH', u'META_COLS', u'META_INFO', u'META_SUB_INFO', u'MIDETEC', u'MINODE', u'MISECT', u'MISUBPATH', u'MISYS', u'MITURN', u'SIM_INFO']


In [30]:
tables

[u'DETEQUIPVEH',
 u'META_COLS',
 u'META_INFO',
 u'META_SUB_INFO',
 u'MIDETEC',
 u'MINODE',
 u'MISECT',
 u'MISUBPATH',
 u'MISYS',
 u'MITURN',
 u'SIM_INFO']

In [26]:
columnsQuery = "PRAGMA table_info(%s)" % tables[0]
#This query is just for column info



cursor.execute(columnsQuery)
columns=cursor.fetchall()
for column in columns:
    print column
numberOfColumns = len(cursor.fetchall())

(0, u'did', u'INTEGER', 0, None, 0)
(1, u'oid', u'INTEGER', 0, None, 0)
(2, u'timedet', u'FLOAT', 0, None, 0)
(3, u'idveh', u'INTEGER', 0, None, 0)
(4, u'vehtype', u'INTEGER', 0, None, 0)
(5, u'idptline', u'INTEGER', 0, None, 0)
(6, u'speed', u'FLOAT', 0, None, 0)
(7, u'headway', u'FLOAT', 0, None, 0)


In [34]:
#We take a look at the table misect, which include the link stats data we need
sectionQuery="SELECT * FROM MISECT"
cursor.execute(sectionQuery)
rows = cursor.fetchall()


In [46]:
nameQuery="SELECT * FROM MISECT"
cursor.execute(nameQuery)
rowName = map(lambda x: x[0], cursor.description)
#rowName=cursor.fetchall()
print rowName

['did', 'oid', 'eid', 'sid', 'ent', 'flow', 'flow_D', 'count', 'count_D', 'input_flow', 'input_flow_D', 'input_count', 'input_count_D', 'ttime', 'ttime_D', 'dtime', 'dtime_D', 'dtimeVQ', 'dtimeVQ_D', 'dtimeTtime', 'dtimeTtime_D', 'speed', 'speed_D', 'spdh', 'spdh_D', 'flow_capacity', 'flow_capacity_D', 'density', 'density_D', 'qmean', 'qmean_D', 'qmax', 'qmax_D', 'qvmean', 'qvmean_D', 'qvmax', 'qvmax_D', 'qvnbvehs', 'travel', 'travel_D', 'traveltime', 'traveltime_D', 'lane_changes', 'lane_changes_D', 'stime', 'stime_D', 'nstops', 'nstops_D']


In [59]:
rows=np.array(rows)
rows.shape

(87906L, 48L)

In [195]:
#need sid=0 (all vehicles), and 
#ent=1, time interval
#check if time interval 1 equals to time interval 0
#play around for a little bit, if everything works fine, then process the final data in python. Otherwise
#save it to matlab

#column 

#np.where((rows[:,3]==1) & (rows[:,4]==0))
stat=rows[(rows[:,3]==0) & (rows[:,4]==1)]
#stat=rows

In [196]:
flowIdx=rowName.index('flow') #flow: mean flow (veh/h)
countIdx=rowName.index('count') #vehicle counts (veh)
ttimeIdx=rowName.index('ttime') #mean travel time (s)
speedIdx=rowName.index('speed')#mean speed (km/h)
densityIdx=rowName.index('density')#density (veh/km)
qmeanIdx=rowName.index('qmean')#mean queue length (veh)
sectionIdx=rowName.index('oid')#sectionId

In [197]:
87906/13/2 #13 vehicle types and 2 intervals (1 aggregated interval) 
#need to check if we have 13 vehicle types, 3381 links, really
#checked. The total number of links is 3381

3381

In [198]:
len(np.unique(rows[:,1]))

3381

In [199]:
#make a np array that include the index of flow, count,etc...
statIdx=np.array([sectionIdx,flowIdx, countIdx,ttimeIdx,speedIdx,qmeanIdx])



In [200]:
nonNullSecIdx=np.where(stat[:,flowIdx]>0)#filer out
np.array(nonNullSecIdx).shape[1]

3249L

In [201]:
secStat=stat[np.array(nonNullSecIdx),:][0]
secStat

array([[16021502, 367, u'4476', ..., 8.805863453400498,
        0.10331632653061225, -1.0],
       [16021502, 368, u'4476', ..., 36.671321423426114,
        0.4748427672955975, -1.0],
       [16021502, 369, u'4490', ..., 0.0, 0.0, -1.0],
       ..., 
       [16021502, 15763164, None, ..., 0.3150571419471383,
        0.7333333333333333, -1.0],
       [16021502, 15763166, None, ..., 0.25772157965221937,
        0.8333333333333334, -1.0],
       [16021502, 15763172, None, ..., 0.0, 0.0, -1.0]], dtype=object)

In [235]:
#since a small portion of sections have no flow, there is no need to filter them out
#rewrite scripts

secStatRep=stat[:,statIdx]
secStatRep.shape

(3381L, 6L)

In [215]:
stat[0]

array([16021502, 367, u'4476', 0, 1, 784.0, -1.0, 784.0, -1.0, 785.0, -1.0,
       785.0, -1.0, 11.679054608690283, 9.789390387418125,
       4.26069430788263, 9.581389477736767, 4.26069430788263,
       9.581389477736767, 36.481500006963614, -1.0, 60.29632065661884,
       16.46661034190842, 50.149417887912016, 22.557953524552925,
       0.38487279724959944, -1.0, 6.628523135041511, -1.0,
       0.13639351851851853, -1.0, 5.0, -1.0, 0.0, -1.0, 0.0, -1.0, 0.0,
       127.52078238024973, -1.0, 9156.378813213181, -1.0,
       0.23341836734693877, -1.0, 1.7985538753731813, 8.805863453400498,
       0.10331632653061225, -1.0], dtype=object)

In [216]:
print rowName

['did', 'oid', 'eid', 'sid', 'ent', 'flow', 'flow_D', 'count', 'count_D', 'input_flow', 'input_flow_D', 'input_count', 'input_count_D', 'ttime', 'ttime_D', 'dtime', 'dtime_D', 'dtimeVQ', 'dtimeVQ_D', 'dtimeTtime', 'dtimeTtime_D', 'speed', 'speed_D', 'spdh', 'spdh_D', 'flow_capacity', 'flow_capacity_D', 'density', 'density_D', 'qmean', 'qmean_D', 'qmax', 'qmax_D', 'qvmean', 'qvmean_D', 'qvmax', 'qvmax_D', 'qvnbvehs', 'travel', 'travel_D', 'traveltime', 'traveltime_D', 'lane_changes', 'lane_changes_D', 'stime', 'stime_D', 'nstops', 'nstops_D']


In [238]:
saveFileName='C:/MIT/Lab/LinsenChong/SANDAG/Database/test.txt'
np.savetxt(saveFileName,secStatRep,fmt=['%i', '%i', '%i', '%f', '%f', '%f'])

In [226]:
#db.commit() #commit the changes
db.close() #close the database

In [ ]:
#next step. write an iterative script that includes: update database next, Update savetxt file name, etc..)